In [0]:
%%time
import os
import sys
import csv

import numpy as np
import pandas as pd

sys.path.append("../") 
from bert_sklearn import BertRegressor
from bert_sklearn import load_model

Wall time: 2.73 s


In [0]:
dev_file='test.tsv'

dev = pd.read_csv(dev_file, sep='\t', encoding='utf8', keep_default_na=False)
dev.columns=['id','tweet']

X_dev= dev['tweet']

label = dev['id']

In [0]:
def get_sst_test_data(train_file ='train.tsv'):
    
    train = pd.read_csv(train_file, sep='\t', encoding='utf8', keep_default_na=False)
    train.columns=['tweet','lab']
    
    X_train = train['tweet']
    y_train = train['lab']
    

    return X_train,y_train

In [0]:
X_train,y_train =  get_sst_test_data()

In [0]:
# define model
# 0.5597471491
model = BertRegressor('bert-base-cased')
model.validation_fraction = 0.0
model.learning_rate = 3e-5 
model.gradient_accumulation_steps = 1
model.max_seq_length = 64
model.train_batch_size = 16
model.eval_batch_size = 8
model.epochs = 2

Building sklearn text regressor...


In [0]:
# fit
model.fit(X_train,y_train)

Loading bert-base-cased model...


100%|█████████████████████████████████████████████████████████████████████████████████████████| 361/361 [00:00<?, ?B/s]


Defaulting to linear classifier/regressor
Loading Pytorch checkpoint
train data size: 12071, validation data size: 0


Training  : 100%|██████████████████████████████████████████████████████| 755/755 [2:40:21<00:00, 10.77s/it, loss=0.365]


BertRegressor(bert_config_json=None, bert_model='bert-base-cased',
              bert_vocab=None, do_lower_case=False, epochs=2, eval_batch_size=8,
              fp16=False, from_tf=False, gradient_accumulation_steps=1,
              ignore_label=None, label_list=None, learning_rate=3e-05,
              local_rank=-1, logfile='bert_sklearn.log', loss_scale=0,
              max_seq_length=64, num_mlp_hiddens=500, num_mlp_layers=0,
              random_state=42, restore_file=None, train_batch_size=16,
              use_cuda=True, validation_fraction=0.0, warmup_proportion=0.1)

In [0]:
y_pred = model.predict(X_dev)

print(y_pred)

Predicting: 100%|████████████████████████████████████████████████████████████████████| 378/378 [10:32<00:00,  1.63s/it]


[0.92944396 0.92945045 0.92943996 ... 0.9294476  0.92944837 0.92944664]


In [0]:
# prepare submission dataframe
BertReg = pd.DataFrame({'id':label, 'pred':y_pred})

# write predictions to a CSV file
BertReg.to_csv("task-1-output.csv", index=False)
